# Setups, Installations and Imports

In [1]:
%%capture 
!pip install tensorflow
!pip install tensorflow-addons
!pip install bayesian-optimization

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import resnet50

import tensorflow_addons as tfa
import sys

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1110309709460546231,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6292504576
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13071288725251095023
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [4]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
except:
    IN_COLAB = False

In [5]:
import os
import shutil
os.environ["TF_DETERMINISTIC_OPS"] = "1"
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import io
import itertools
from functools import partial
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, recall_score
from tqdm.notebook import tqdm_notebook
from ipywidgets import IntProgress

from sklearn.model_selection import KFold, StratifiedKFold

from bayes_opt import BayesianOptimization

from sklearn.preprocessing import LabelBinarizer


# Download and Prepare Dataset

In [6]:
# Insert the directoryimport sys
if IN_COLAB:
    '''this is the exact path of the mounted direttory. in order to set it, right click on colab notebooks folder and copy path, and insert below '''
    COLAB_NOTEBOOKES_PATH = "/content/drive/MyDrive/Colab Notebooks"
    sys.path.insert(0,COLAB_NOTEBOOKES_PATH)  

import Ensembles_prepare_datasets


The syntax of the command is incorrect.
'touch' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'cat' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


#### Dataloader

In [7]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

def preprocess_image(image, label):
  img = tf.cast(image, tf.float32)
  img = img/255.

  return img, label


def get_loaders(x_train, y_train, x_test, y_test):
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    trainloader = (
        trainloader
        .shuffle(1024)
        .map(preprocess_image, num_parallel_calls=AUTO)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

    testloader = (
        testloader
        .map(preprocess_image, num_parallel_calls=AUTO)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )
    return trainloader, testloader

In [8]:
def clean_folder(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

ROOT_PATH = './'
## SmallCNN models with different initialization
models_dir = 'IndependentSolutions'
MODEL_PATH = ROOT_PATH+'IndependentSolutions/'

if models_dir not in os.listdir(ROOT_PATH):
    os.mkdir(models_dir)



# Model

In [9]:
def Model1(num_labels, img_shape, color = 3):
  inputs = keras.layers.Input(shape=(img_shape, img_shape, color))

  x = keras.layers.Conv2D(16, (3,3), padding='same')(inputs)
  x = keras.activations.relu(x)
  x = keras.layers.MaxPooling2D(2, strides=2)(x)

  x = keras.layers.Conv2D(32,(3,3), padding='same')(x)
  x = keras.activations.relu(x)
  x = keras.layers.MaxPooling2D(2, strides=2)(x)

  x = keras.layers.Conv2D(32,(3,3), padding='same')(x)
  x = keras.activations.relu(x)
  x = keras.layers.MaxPooling2D(2, strides=2)(x)

  x = keras.layers.GlobalAveragePooling2D()(x)
  x = keras.layers.Dense(32, activation='relu')(x)
  x = keras.layers.Dropout(0.1)(x)
  
  outputs = keras.layers.Dense(num_labels, activation='softmax')(x)

  return keras.models.Model(inputs=inputs, outputs=outputs)

def Model():
    if color:
        return Model1(num_labels, img_shape,color)
    else:
        return Model1(num_labels, img_shape)

# Init hyper parameters

In [10]:

learning_rate_range = [0.001,0.1]
epsilon_range = [1e-08,1e-05]


# Callbacks

#### LR Scheduler

In [11]:

def lr_schedule(epoch,lr):
    if (epoch >= 0) & (epoch < 9):
        return lr
    elif (epoch >= 9) & (epoch < 19):
        return lr/2
    elif (epoch >= 19) & (epoch < 29):
        return lr/4
    else:
        return lr/8

#lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lr_schedule(epoch), verbose=True)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

# Independent Solutions

In [12]:
# Saving 5 independent solutions i.e, 5 different inits.
# Will save last model snapshot. 

def run_independent_solutions():

    EPOCHS = 10
    SAVE_PATH = 'IndependentSolutions/'

    for iter in tqdm_notebook(range(5)):
      # initialize weights
      keras.backend.clear_session()
      model = Model()
      #optimizer define with name
      opt = tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-07, name='Adam'+str(iter))
      # compile
      model.compile(opt, 'sparse_categorical_crossentropy', metrics=['accuracy'])
      # train
      _ = model.fit(trainloader,
                  epochs=EPOCHS,
                  validation_data=testloader,
                  callbacks=[lr_callback],
                  verbose=0)
      # save final model snapshot
      model.save(SAVE_PATH+'smallcnn_independent_model_{}.h5'.format(iter))
      # evaluate
      loss, accuracy = model.evaluate(testloader, verbose=0)
      print("Test Error Rate: ", round((1-accuracy)*100, 2), '% | Iteration No: ', iter)

def run_independent_solutions(lr,ep,trainloader,testloader):

    EPOCHS = 10
    SAVE_PATH = 'IndependentSolutions/'

    for iter in tqdm_notebook(range(5)):
      # initialize weights
      keras.backend.clear_session()
      model = Model()
      #optimizer define with name
      opt = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=ep, name='Adam'+str(iter))
      # compile
      model.compile(opt, 'sparse_categorical_crossentropy', metrics=['accuracy'])
      # train
      _ = model.fit(trainloader,
                  epochs=EPOCHS,
                  validation_data=testloader,
                  callbacks=[lr_callback],
                  verbose=0)
      # save final model snapshot
      model.save(SAVE_PATH+'smallcnn_independent_model_{}.h5'.format(iter))
      # evaluate
      loss, accuracy = model.evaluate(testloader, verbose=0)
      print("Test Error Rate: ", round((1-accuracy)*100, 2), '% | Iteration No: ', iter)
#run_independent_solutions()

def run_independent_solutions_additive(lr,ep,trainloader,testloader,f_x_test,f_y_test):

    EPOCHS = 10
    SAVE_PATH = 'IndependentSolutions/'
    clean_folder(SAVE_PATH)
    accuracy_list = [0]
    improvments = []
    for iter in tqdm_notebook(range(10)):
        # initialize weights
        keras.backend.clear_session()
        model = Model()
        #optimizer define with name
        opt = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=ep, name='Adam'+str(iter))
        # compile
        model.compile(opt, 'sparse_categorical_crossentropy', metrics=['accuracy'])
        # train
        _ = model.fit(trainloader,
                  epochs=EPOCHS,
                  validation_data=testloader,
                  callbacks=[lr_callback],
                  verbose=0)
        # save final model snapshot
        model.save(SAVE_PATH+'smallcnn_independent_model_{}.h5'.format(iter))
        # evaluate
        loss, accuracy = model.evaluate(testloader, verbose=0)
        print("Test Error Rate: ", round((1-accuracy)*100, 2), '% | Iteration No: ', iter)

        model_ckpts = os.listdir(MODEL_PATH)
        members = [tf.keras.models.load_model(MODEL_PATH + model_ckpts[i]) 
                            for i in range(len(model_ckpts))]
        yhat = ensemble_predictions(members,f_x_test)
        accuracy = 100*accuracy_score(f_y_test, yhat)
        accuracy_list.append(accuracy)
        #improvments = [j-i for i, j in zip(accuracy_list[:-1], accuracy_list[1:])]
        improvments.append(accuracy_list[-1]-accuracy_list[-2])
        print(accuracy_list)
        print(improvments)
        if(len(improvments) > 3):
            if(improvments[-1] < np.mean(improvments[-4:-1])*1.00):
                print("found best amount of models: " + str(len(improvments)))
                return
            
        
    print('did not find amount of models better then maximum')
        
        
#run_independent_solutions()

In [13]:
arr = [1,2,3,4,5,6,7,8,9]
for i, j in zip(arr[:-1], arr[1:]):
    print(i,j)
    
print(arr[-4:-1])
print(arr[-1]-arr[-2])

1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
[6, 7, 8]
1


# Ensembles


# Deep Ensembles

In [14]:
# make an ensemble prediction for multi-class classification
def ensemble_predictions(members):
	# make predictions
	yhats = [model.predict(x_test/255.) for model in members]
	yhats = np.array(yhats)
	# sum across ensemble members
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	result = np.argmax(summed, axis=1)
	return result
 
# evaluate a specific number of members in an ensemble
def evaluate_n_members(members):
    accuracy_list = []
    for n_members in tqdm_notebook(range(1, len(members))):
        # select a subset of members
        subset = members[:n_members]
        print(len(subset))
        # make prediction
        yhat = ensemble_predictions(subset)
        # calculate accuracy
        accuracy_list.append(100*accuracy_score(y_test, yhat))

    return accuracy_list

def ensemble_predictions(members, test):
	# make predictions
	yhats = [model.predict(test/255.) for model in members]
	yhats = np.array(yhats)
	# sum across ensemble members
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	result = np.argmax(summed, axis=1)
	return result

In [15]:
# Run the ensembles for plot
def run_ensembles_and_plot():
    members = [tf.keras.models.load_model(MODEL_PATH + model_ckpts[i]) 
                            for i in range(len(model_ckpts))]
    
    single_model = np.random.choice(members)
    accuracy_single_model = single_model.evaluate(testloader)[1] * 100
    
    accuracy_list = evaluate_n_members(members)
    
    accuracy_list.insert(0, accuracy_single_model)
    rng = [i for i in range(0, len(members))]
    
    plt.figure(figsize=(9,8))
    plt.plot(rng, accuracy_list, label='deep ensemble')
    plt.plot(rng, [accuracy_single_model]*len(rng), '--', label='single model')
    plt.title("Test accuracy as a function of ensemble size")
    plt.xlabel("Ensemble size")
    plt.ylabel("Test accuracy")
    plt.legend();
    plt.savefig('ensemble_func.png')
    
#run_ensembles_and_plot()

In [16]:
# Bayesian Optimization with k fold validation 
def iner_kfold(x_train, y_train, lr, ep):
    kf = KFold(n_splits = 3, random_state=3, shuffle=True)
    accuracy_list = []
    for train_index, test_index in kf.split(x_train):
        f_x_train, f_x_test = x_train[train_index], x_train[test_index]
        f_y_train, f_y_test = y_train[train_index], y_train[test_index]
        trainloader, testloader = get_loaders(f_x_train, f_y_train, f_x_test, f_y_test)
        run_independent_solutions(lr, ep,trainloader,testloader)
        model_ckpts = os.listdir(MODEL_PATH)
        members = [tf.keras.models.load_model(MODEL_PATH + model_ckpts[i]) 
                            for i in range(len(model_ckpts))]
        yhat = ensemble_predictions(members,f_x_test)
        accuracy = 100*accuracy_score(f_y_test, yhat)
        accuracy_list.append(accuracy)
    print('accuracy:', np.mean(accuracy_list), '+/-', np.std(accuracy_list))
    return np.mean(accuracy_list)

def kfold_opt(x_train, y_train):
    print("finding best hyper parameters")
    fit_with_partial = partial(iner_kfold, x_train, y_train)
    pbounds = { 'lr': (0.0001,0.01), 'ep': (1e-8,1e-5)}

    optimizer = BayesianOptimization(
        f=fit_with_partial,
        pbounds=pbounds,
        verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
        random_state=1,
    )

    optimizer.maximize(init_points=10, n_iter=10)


    for i, res in enumerate(optimizer.res):
        print("Iteration {}: \n\t{}".format(i, res))

    print(optimizer.max)
    return optimizer.max['params']['lr'], optimizer.max['params']['ep']




In [17]:
#claculating the result TPR and FPR
def tpr_fpr(y_true, y_prediction):

    cnf_matrix = confusion_matrix(y_true, y_prediction)
   
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float).sum()
    FN = FN.astype(float).sum()
    TP = TP.astype(float).sum()
    TN = TN.astype(float).sum()

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    return TPR, FPR

In [18]:
#writing the results to a file
def write_res(name, algo, cros_val, hp_lr, hp_ep, accu, tpr, fpr, preci, auc, train_time):
    file_name = name + '-' + algo + ".csv"
    print(file_name)
    f = open(file_name, "a")
    f.write(name+','+algo+','+str(cros_val)+','+str(hp_lr)+','+str(hp_ep)+','
            +str(accu)+','+str(tpr)+','+str(fpr)+','+str(preci)+','+str(auc)+','+str(train_time)+'\n')
    f.close()

In [19]:
#get AUC of multiclass results
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

In [20]:
# validate preformence result with kfold 
def kfold_val():
    f = open(n + "-improved_algo.csv", "w")
    f.write('name, algo, cros_val, hp_lr, hp_ep, accu, tpr, fpr, preci, auc, train_time\n')
    f.close()
    kf = KFold(n_splits=10, random_state=3, shuffle=True) #should be 10
    start1 = time.time()
    lr=0.0005
    ep=1e-07
    accuracy_list = []
    training_times = []
    first_fold = False#True
    cros_iter = 1
    for train_index, test_index in kf.split(x):
        print("TRAIN:", train_index, "TEST:", test_index)
        f_x_train, f_x_test = x[train_index], x[test_index]
        f_y_train, f_y_test = y[train_index], y[test_index]
        start2 = time.time()
        if first_fold:
            #find best hyperparameters to use
            lr, ep = kfold_opt(f_x_train, f_y_train)
            first_fold = False
        
        trainloader, testloader = get_loaders(f_x_train, f_y_train, f_x_test, f_y_test) 
        run_independent_solutions_additive(lr,ep,trainloader,testloader,f_x_test,f_y_test)
        end2 = time.time()
        model_ckpts = os.listdir(MODEL_PATH)
        members = [tf.keras.models.load_model(MODEL_PATH + model_ckpts[i]) 
                            for i in range(len(model_ckpts))]
        yhat = ensemble_predictions(members,f_x_test)
        ##### infer results and saving
        accuracy = 100*accuracy_score(f_y_test, yhat)
        TPR, FPR = tpr_fpr(f_y_test, yhat)
        precision = precision_score(f_y_test, yhat, average='weighted')
        auc = multiclass_roc_auc_score(f_y_test,yhat)
        accuracy_list.append(accuracy)
        train_time = end2 - start2
        training_times.append(train_time)
        print("accuracy: {}".format(accuracy))
        write_res(n, 'improved_algo', cros_iter, lr, ep, accuracy, TPR, FPR, precision, auc, train_time)
        cros_iter = cros_iter + 1
    end1 = time.time()
    print('accuracy:', np.mean(accuracy_list), '+/-', np.std(accuracy_list))
    print("avarage accuracy: {}".format(np.average(accuracy_list)) )   
    print("Network takes {:.3f} minuts to fold validate".format((end1 - start1)/60))

#kfold_val()

In [21]:
#model_list = ['cifar10', 'cifar100', 'mnist', 'flowers102', 'monkeys10', 'intel','cifar10_2', 'cifar100_2', 'mnist_2', 'flowers102_2', 'monkeys10_2', 'intel_2']

#model_list = ['cifar100', 'cifar100_2', 'flowers102', 'flowers102_2']
model_list = ['cifar10']
color = 0
for n in model_list:
    if n == 'cifar10':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_cifar10()
    elif n == 'cifar10_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_cifar10()
    elif n == 'cifar100':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_cifar100()
    elif n == 'cifar100_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_cifar100()
    elif n == 'mnist':
        x,x2,y,y2,num_labels,img_shape,color = Ensembles_prepare_datasets.load_mnist()
    elif n == 'mnist_2':
        x1,x,y1,y,num_labels,img_shape,color = Ensembles_prepare_datasets.load_mnist()
    elif n == 'flowers102':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_flowers102()
    elif n == 'flowers102_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_flowers102()
        
    
    elif n == 'monkeys10':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_monkeys10()
    elif n == 'monkeys10_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_monkeys10()
    elif n == 'intel':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_intel()
    elif n == 'intel_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_intel()
    kfold_val()

TRAIN: [    0     1     2 ... 29996 29998 29999] TEST: [    5    33    37 ... 29967 29978 29997]


  0%|          | 0/10 [00:00<?, ?it/s]

Test Error Rate:  57.23 % | Iteration No:  0
[0, 42.766666666666666]
[42.766666666666666]
Test Error Rate:  60.37 % | Iteration No:  1
[0, 42.766666666666666, 41.96666666666667]
[42.766666666666666, -0.7999999999999972]
Test Error Rate:  56.33 % | Iteration No:  2
[0, 42.766666666666666, 41.96666666666667, 43.43333333333334]
[42.766666666666666, -0.7999999999999972, 1.4666666666666686]
Test Error Rate:  58.13 % | Iteration No:  3
[0, 42.766666666666666, 41.96666666666667, 43.43333333333334, 43.56666666666666]
[42.766666666666666, -0.7999999999999972, 1.4666666666666686, 0.13333333333332575]
found best amount of models: 4
accuracy: 43.56666666666666
cifar10-improved_algo.csv
TRAIN: [    0     1     2 ... 29997 29998 29999] TEST: [   11    25    26 ... 29980 29982 29983]


  0%|          | 0/10 [00:00<?, ?it/s]

Test Error Rate:  58.33 % | Iteration No:  0
[0, 41.66666666666667]
[41.66666666666667]
Test Error Rate:  55.47 % | Iteration No:  1
[0, 41.66666666666667, 44.06666666666666]
[41.66666666666667, 2.3999999999999915]
Test Error Rate:  55.1 % | Iteration No:  2
[0, 41.66666666666667, 44.06666666666666, 45.13333333333333]
[41.66666666666667, 2.3999999999999915, 1.06666666666667]
Test Error Rate:  58.07 % | Iteration No:  3
[0, 41.66666666666667, 44.06666666666666, 45.13333333333333, 44.5]
[41.66666666666667, 2.3999999999999915, 1.06666666666667, -0.6333333333333329]
found best amount of models: 4
accuracy: 44.5
cifar10-improved_algo.csv
TRAIN: [    0     1     2 ... 29997 29998 29999] TEST: [   13    19    22 ... 29984 29986 29990]


...

Test Error Rate:  55.63 % | Iteration No:  0
[0, 44.36666666666667]
[44.36666666666667]
Test Error Rate:  54.27 % | Iteration No:  1
[0, 44.36666666666667, 46.96666666666667]
[44.36666666666667, 2.6000000000000014]
Test Error Rate:  55.17 % | Iteration No:  2
[0, 44.36666666666667, 46.96666666666667, 47.53333333333333]
[44.36666666666667, 2.6000000000000014, 0.5666666666666629]
Test Error Rate:  55.63 % | Iteration No:  3
[0, 44.36666666666667, 46.96666666666667, 47.53333333333333, 47.333333333333336]
[44.36666666666667, 2.6000000000000014, 0.5666666666666629, -0.19999999999999574]
found best amount of models: 4
accuracy: 47.333333333333336
cifar10-improved_algo.csv
TRAIN: [    0     1     5 ... 29997 29998 29999] TEST: [    2     3     4 ... 29992 29995 29996]


  0%|          | 0/10 [00:00<?, ?it/s]

Test Error Rate:  59.33 % | Iteration No:  0
[0, 40.666666666666664]
[40.666666666666664]
Test Error Rate:  58.13 % | Iteration No:  1
[0, 40.666666666666664, 43.0]
[40.666666666666664, 2.3333333333333357]
Test Error Rate:  58.03 % | Iteration No:  2
[0, 40.666666666666664, 43.0, 43.56666666666666]
[40.666666666666664, 2.3333333333333357, 0.5666666666666629]
Test Error Rate:  56.83 % | Iteration No:  3
[0, 40.666666666666664, 43.0, 43.56666666666666, 43.96666666666667]
[40.666666666666664, 2.3333333333333357, 0.5666666666666629, 0.4000000000000057]
found best amount of models: 4
accuracy: 43.96666666666667
cifar10-improved_algo.csv
TRAIN: [    0     1     2 ... 29995 29996 29997] TEST: [   12    32    44 ... 29993 29998 29999]


  0%|          | 0/10 [00:00<?, ?it/s]

Test Error Rate:  61.37 % | Iteration No:  0
[0, 38.63333333333333]
[38.63333333333333]
Test Error Rate:  57.37 % | Iteration No:  1
[0, 38.63333333333333, 41.63333333333333]
[38.63333333333333, 3.0]
Test Error Rate:  56.33 % | Iteration No:  2
[0, 38.63333333333333, 41.63333333333333, 42.699999999999996]
[38.63333333333333, 3.0, 1.0666666666666629]
Test Error Rate:  56.7 % | Iteration No:  3
[0, 38.63333333333333, 41.63333333333333, 42.699999999999996, 43.766666666666666]
[38.63333333333333, 3.0, 1.0666666666666629, 1.06666666666667]
found best amount of models: 4
accuracy: 43.766666666666666
cifar10-improved_algo.csv
accuracy: 44.27333333333333 +/- 1.2812147014107795
avarage accuracy: 44.27333333333333
Network takes 7.903 minuts to fold validate
